In [4]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

In [5]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")
g.clear()

CREATE CONSTRAINT ON (n:node) ASSERT n.id IS UNIQUE
MATCH (n:node)
OPTIONAL MATCH (n)-[r]-()
DELETE n, r



In [6]:
g.add_nodes_from([
    ("n1", {"name": "Nils"}),
    ("n2", {"acmid": 220})
])
g.add_edge("n1", "n2", {"contact": True})


OPTIONAL MATCH (same_id_node:node) 
WHERE same_id_node.id = 'n1' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['n1' + same_id_node.count]
	ELSE ['n1'] END AS new_id_n1 
		CREATE (n1:node { id : new_id_n1 }) 
SET n1.name=['Nils'] WITH new_id_n1, n1 OPTIONAL MATCH (same_id_node:node) 
WHERE same_id_node.id = 'n2' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['n2' + same_id_node.count]
	ELSE ['n2'] END AS new_id_n2 
		CREATE (n2:node { id : new_id_n2 }) 
SET n2.acmid=[220] WITH n2, new_id_n2 RETURN n2, new_id_n2

MATCH (n1:node { id : 'n1'}) , (n2:node { id : 'n2'})  MERGE (n1)-[new_edge:edge]->(n2)
SET new_edge.contact=[True] 
